Les imports

In [38]:
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

from keras.preprocessing.image import ImageDataGenerator

import math
import time

from commun.CommunFonction import *
from commun.CONSTANTE import *

Paramètres Généraux

In [39]:
dir = "../commun/datasets/CNN"
folderTrain = dir + "/" + "training_set"
folderTest = dir + "/" + "test_set"

Comptage du nombre d'images

In [40]:
nbImageTrain, nbImageTest = comptageTot(folderTrain, folderTest)

Construction de l'IA

In [41]:
def buildModel():
    model = Sequential()
    model.add(Convolution2D(filters=NUMBER_FILTRE, kernel_size=KERNEL_SIZE, strides=STRIDE_CONVOLUTION, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
                            activation=FUNCTION_ACTIVATION))

    model.add(MaxPooling2D(pool_size=SIZE_MATRICE_POOLING, strides=STRIDE_POOLING))

    nbrLayer = 1
    multiplicateur = 1
    for i in range(NUMBER_LAYER_POOL_CONV - 1):
        model.add(Convolution2D(filters=NUMBER_FILTRE * multiplicateur, kernel_size=KERNEL_SIZE, strides=STRIDE_CONVOLUTION,
                                activation=FUNCTION_ACTIVATION))

        model.add(MaxPooling2D(pool_size=SIZE_MATRICE_POOLING, strides=STRIDE_POOLING))
        nbrLayer += 1
        multiplicateur = math.floor((nbrLayer / 2) + 1)

    model.add(Flatten())

    for j in range(NUMBER_HIDDEN_LAYER):
        model.add(Dense(units=NUMBER_NEURONE_HIDDEN_LAYER, activation=FUNCTION_ACTIVATION))

    model.add(Dense(units=NUMBER_NEURONE_OUTPUT, activation=FUNCTION_OUTPOUT))

    model.compile(optimizer=OPTIMIZER, loss=FUNCTION_LOSS, metrics=METRIC)

    return model



CNN = buildModel()

Initialisation des jeux des données

In [42]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train = train_datagen.flow_from_directory(folderTrain,
                                          target_size=IMAGE_SIZE_TUPLE,
                                          batch_size=BATCH_SIZE,
                                          class_mode=CLASS_MODE)

test = test_datagen.flow_from_directory(folderTrain,
                                          target_size=IMAGE_SIZE_TUPLE,
                                          batch_size=BATCH_SIZE,
                                          class_mode=CLASS_MODE)

Found 8005 images belonging to 2 classes.
Found 8005 images belonging to 2 classes.


Entrainement de l'IA

In [43]:
saves = CNN.fit(
    train,
    steps_per_epoch=math.ceil(nbImageTrain / BATCH_SIZE),
    epochs=NUMBER_EPOCH,
    validation_data=test,
    validation_steps=math.ceil(nbImageTest / BATCH_SIZE),
    verbose=1)

Epoch 1/15
 64/251 [======>.......................] - ETA: 52s - loss: 0.6892 - acc: 0.5205 

KeyboardInterrupt: 

Sauvegarde des Information lier a l'apprentissage

In [ ]:
writeIntoFile(saves.history['val_acc'],saves.history['acc'],"keras_CNN")